In [11]:
import pygame  
import sys  
import copy 
import random 
import time  

pygame.init()  

# Window parameters  
scale = 15  
# Game stats  
score = 0  
level = 0  
SPEED = 10  

# Food parameters
food_x = 10  
food_y = 10  
food_weight = 1  # Default food weight (score value)
food_timer = 0   # Timer for food disappearance
FOOD_LIFETIME = 5  # Food will disappear after 5 seconds

# Game window  
display = pygame.display.set_mode((500, 500))  
pygame.display.set_caption("Extended Snake Game")  
clock = pygame.time.Clock()  

# Colors  
background_top = (170, 51, 106)  
background_bottom = (0, 0, 0)  
snake_colour = (255, 137, 0)  
food_colour = (random.randint(1, 255), random.randint(1, 255), random.randint(1, 255))  
snake_head = (255, 247, 0)  
font_colour = (255, 255, 255)  
defeat_colour = (255, 0, 0)  
timer_warning_colour = (255, 50, 50)  # Color when food is about to disappear

class Snake:  
    def __init__(self, x_start, y_start):  
        # Initial position  
        self.x = x_start  
        self.y = y_start  
        # Size  
        self.w = 15  
        self.h = 15  
        # Movement direction  
        self.x_dir = 1  
        self.y_dir = 0  
        # Position history  
        self.history = [[self.x, self.y]]  
        # Length  
        self.length = 1  

    def reset(self):  
        """Reset the snake to starting position"""
        self.x = 500 / 2 - scale  
        self.y = 500 / 2 - scale  
        self.w = 15  
        self.h = 15  
        self.x_dir = 1  
        self.y_dir = 0  
        self.history = [[self.x, self.y]]  
        self.length = 1  

    def show(self):  
        """Draw the snake on the screen"""
        for i in range(self.length):  
            if not i == 0:  
                pygame.draw.rect(display, snake_colour, (self.history[i][0], self.history[i][1], self.w, self.h))  
            else:  
                pygame.draw.rect(display, snake_head, (self.history[i][0], self.history[i][1], self.w, self.h))  

    def check_eaten(self):  
        """Check if snake ate the food"""
        if abs(self.history[0][0] - food_x) < scale and abs(self.history[0][1] - food_y) < scale:  
            return True  

    def check_level(self):  
        """Check if level should increase"""
        global level  
        if self.length % 5 == 0:  
            return True  

    def grow(self):  
        """Increase snake length"""
        self.length += 1  
        self.history.append(self.history[self.length - 2])  

    def death(self):  
        """Check if snake collided with itself"""
        i = self.length - 1  
        while i > 0:  
            if abs(self.history[0][0] - self.history[i][0]) < self.w and abs(self.history[0][1] - self.history[i][1]) < self.h and self.length > 2:  
                return True  
            i -= 1  

    def update(self):  
        """Update snake position"""
        i = self.length - 1  
        while i > 0:  
            self.history[i] = copy.deepcopy(self.history[i - 1])  
            i -= 1  
        self.history[0][0] += self.x_dir * scale  
        self.history[0][1] += self.y_dir * scale  

class Food:  
    def new_location(self):  
        """Generate new food with random position, weight and color"""
        global food_x, food_y, food_weight, food_colour, food_timer
        
        food_x = random.randrange(1, int(500 / scale) - 1) * scale  
        food_y = random.randrange(1, int(500 / scale) - 1) * scale  
        
        # Random weight between 1-5 (affects score)
        food_weight = random.randint(1, 5)
        
        # Generate color based on weight (heavier food = more red)
        red_component = min(255, 50 + food_weight * 40)
        green_component = max(0, 255 - food_weight * 40)
        food_colour = (red_component, green_component, random.randint(50, 200))
        
        # Reset the disappearance timer
        food_timer = time.time()

    def show(self):  
        """Draw the food on the screen with timer indicator"""
        # Calculate time left before disappearance
        time_left = FOOD_LIFETIME - (time.time() - food_timer)
        
        # Flash red when food is about to disappear (last 3 seconds)
        if time_left < 3 and int(time.time() * 2) % 2 == 0:  # Blinking effect
            current_color = timer_warning_colour
        else:
            current_color = food_colour
            
        pygame.draw.rect(display, current_color, (food_x, food_y, scale, scale))
        
        # Show food weight (score value) as text
        font = pygame.font.SysFont(None, 20)
        weight_text = font.render(str(food_weight), True, (255, 255, 255))
        display.blit(weight_text, (food_x + scale//2 - 5, food_y + scale//2 - 5))

def show_score():  
    """Display current score"""
    font = pygame.font.SysFont(None, 20)  
    text = font.render("Score: " + str(score), True, font_colour)  
    display.blit(text, (scale, scale))  

def show_level():  
    """Display current level"""
    font = pygame.font.SysFont(None, 20)  
    text = font.render("Level: " + str(level), True, font_colour)  
    display.blit(text, (90 - scale, scale))  

def show_time_left():  
    """Display time remaining before food disappears"""
    time_left = max(0, FOOD_LIFETIME - (time.time() - food_timer))
    font = pygame.font.SysFont(None, 20)  
    text = font.render(f"Food: {int(time_left)}s", True, font_colour)  
    display.blit(text, (180, scale))  

def gameLoop():  
    global score  
    global level  
    global SPEED  
    global food_timer

    snake = Snake(500 / 2, 500 / 2)  
    food = Food()  
    food.new_location()  

    while True:  
        for event in pygame.event.get():  
            if event.type == pygame.QUIT:  
                pygame.quit()  
                sys.exit()  
            if event.type == pygame.KEYDOWN:  
                if event.key == pygame.K_q:  
                    pygame.quit()  
                    sys.exit()  
                if snake.y_dir == 0:  
                    if event.key == pygame.K_UP:  
                        snake.x_dir = 0  
                        snake.y_dir = -1  
                    if event.key == pygame.K_DOWN:  
                        snake.x_dir = 0  
                        snake.y_dir = 1  

                if snake.x_dir == 0:  
                    if event.key == pygame.K_LEFT:  
                        snake.x_dir = -1  
                        snake.y_dir = 0  
                    if event.key == pygame.K_RIGHT:  
                        snake.x_dir = 1  
                        snake.y_dir = 0  

        # Gradient background  
        for y in range(500):  
            color = (  
                background_top[0] + (background_bottom[0] - background_top[0]) * y / 500,  
                background_top[1] + (background_bottom[1] - background_top[1]) * y / 500,  
                background_top[2] + (background_bottom[2] - background_top[2]) * y / 500  
            )  
            pygame.draw.line(display, color, (0, y), (500, y))  

        snake.show()  
        snake.update()  
        
        food.show()  
        show_score()  
        show_level()  
        show_time_left()  

        # Check if food should disappear (timer expired)
        if time.time() - food_timer > FOOD_LIFETIME:
            food.new_location()

        if snake.check_eaten():  
            score += food_weight  # Add food's weight to score
            snake.grow()  
            food.new_location()  

        if snake.check_level():  
            level += 1  
            SPEED += 1  
            snake.grow()  

        if snake.death():  
            score = 0  
            level = 0  
            font = pygame.font.SysFont(None, 100)  
            text = font.render("Game Over!", True, defeat_colour)  
            display.blit(text, (50, 200))  
            pygame.display.update()  
            time.sleep(3)  
            snake.reset()  

        # Screen wrapping  
        if snake.history[0][0] > 500:  
            snake.history[0][0] = 0  

        if snake.history[0][0] < 0:  
            snake.history[0][0] = 500  

        if snake.history[0][1] > 500:  
            snake.history[0][1] = 0  

        if snake.history[0][1] < 0:  
            snake.history[0][1] = 500  

        pygame.display.update()  
        clock.tick(SPEED)  

gameLoop()

SystemExit: 